In [5]:
import numpy as np
import matplotlib.pyplot as plt

from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Druckrohrleitung.Druckrohrleitung_class_file import Druckrohrleitung_class


In [6]:
#define constants

# physics
g       = 9.81                          # gravitational acceleration [m/s²]
rho     = 1000.                          # density of water [kg/m³]

# pipeline
L       = 1000.                         # length of pipeline [m]
D       = 1.                            # pipe diameter [m]
#consider replacing Q0 with a vector be be more flexible in initial conditions
Q0      = 2                            # initial flow in whole pipe [m³/s]
A_pipe  = D**2/4*np.pi                  # pipeline area
v0      = Q0/A_pipe                     # initial flow velocity [m/s]
h_res   = 20.                           # water level in upstream reservoir [m]
n       = 10                            # number of pipe segments in discretization
nt      = 10000                           # number of time steps after initial conditions
f_D     = 0.01                          # Darcy friction factor
c       = 400.                          # propagation velocity of the pressure wave [m/s]
h_pipe  = 300                           # hydraulic head without reservoir [m] 
alpha   = np.arcsin(h_pipe/L)           # Höhenwinkel der Druckrohrleitung 

# derivatives of the pipeline constants
p0 = rho*g*h_res-v0**2*rho/2
dx      = L/n                                   # length of each pipe segment
dt      = dx/c                                  # timestep according to method of characterisitics
nn      = n+1                                   # number of nodes
pl_vec  = np.arange(0,nn*dx,dx)                 # pl = pipe-length. position of the nodes on the pipeline
t_vec   = np.arange(0,nt*dt,dt)                 # time vector
h_vec   = np.arange(0,h_pipe+h_pipe/n,h_pipe/n) # hydraulic head of pipeline at each node

v_init = np.full(nn,Q0/(D**2/4*np.pi))
p_init = (rho*g*(h_res+h_vec)-v_init**2*rho/2)-(f_D*pl_vec/D*rho/2*v_init**2) # ref Wikipedia: Darcy Weisbach


# reservoir
initial_level               = h_res     # m
initial_influx              = 0.        # m³/s
initial_outflux             = Q0        # m³/s
initial_pipeline_pressure   = p0        # Pa 
initial_pressure_unit       = 'Pa'
conversion_pressure_unit    = 'Pa'
area_base                   = 5.      # m²   really large base are to ensure level never becomes < 0
area_outflux                = A_pipe    # m²
critical_level_low          = 0.        # m
critical_level_high         = np.inf    # m

# make sure e-RK4 method of reservoir has a small enough timestep to avoid runaway numerical error
nt_eRK4                     = 1000      # number of simulation steps of reservoir in between timesteps of pipeline              
simulation_timestep         = dt/nt_eRK4



In [7]:
print(pressure_conversion(-np.sum((-v_init**2*rho/2)),'Pa','mWS'))

(3.6368236494728476, 'mWS')


In [8]:
# create objects

V = Ausgleichsbecken_class(area_base,area_outflux,critical_level_low,critical_level_high,simulation_timestep)
V.set_initial_level(initial_level) 
V.set_influx(initial_influx)
V.set_outflux(initial_outflux)
V.pressure, V.pressure_unit = pressure_conversion(initial_pipeline_pressure,input_unit = initial_pressure_unit, target_unit = conversion_pressure_unit)

pipe = Druckrohrleitung_class(L,D,n,alpha,f_D)
pipe.set_pressure_propagation_velocity(c)
pipe.set_number_of_timesteps(nt)
pipe.set_initial_pressure(p_init)
pipe.set_initial_flow_velocity(v_init)

In [9]:
# initialization for timeloop

v_old = v_init.copy()
p_old = p_init.copy()

#vectors to store boundary conditions
v_boundary_res  = np.empty_like(t_vec)
v_boundary_tur  = np.empty_like(t_vec)
p_boundary_res  = np.empty_like(t_vec)
p_boundary_tur  = np.empty_like(t_vec)
level_vec       = np.empty_like(t_vec)
level_vec_2     = np.full([nt_eRK4],initial_level)

v_boundary_res[0]       = v_old[0]
v_boundary_tur[0]       = v_old[-1] # instantaneous closing
# v_boundary_tur[1:]      = 0
v_boundary_tur[0:1000]   = np.linspace(v_old[-1],0,1000) # finite closing time - linear case
p_boundary_res[0]       = p_old[0]
p_boundary_tur[0]       = p_old[-1]
level_vec[0]            = initial_level

v_boundary_tur[1:]  = 0 # instantaneous closing

In [10]:
%matplotlib qt5
# time loop


# fig2,axs2 = plt.subplots(3,1)
# axs2[0].set_title('Pressure distribution in pipeline')
# axs2[1].set_title('Velocity distribution in pipeline')
# axs2[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
# axs2[0].set_ylabel(r'$p$ [mWS]')
# axs2[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
# axs2[1].set_ylabel(r'$p$ [mWS]')
# lo_00, = axs2[0].plot(pl_vec,pressure_conversion(pipe.p_old,'Pa','mWS')[0],marker='.')
# lo_01, = axs2[1].plot(pl_vec,pipe.v_old,marker='.')
# lo_02, = axs2[2].plot(level_vec_2)
# axs2[0].autoscale()
# axs2[1].autoscale()
# axs2[2].autoscale()
# fig2.tight_layout()

# loop through time steps of the pipeline
for it_pipe in range(1,pipe.nt):

# for each pipeline timestep, execute nt_eRK4 timesteps of the reservoir code
    V.pressure  = p_old[0]
    V.outflux   = v_old[0]
    for it_res in range(nt_eRK4):
        V.e_RK_4()
        V.level = V.update_level(V.timestep)
        V.set_volume()
        level_vec_2[it_res] = V.level
        if (V.level < critical_level_low) or (V.level > critical_level_high):
            i_max = it_pipe
            print('broke')
            break
    level_vec[it_pipe] = V.level

    p_boundary_res[it_pipe] = rho*g*V.level-v_old[1]**2*rho/2
    v_boundary_res[it_pipe] = v_old[1]+1/(rho*c)*(p_boundary_res[it_pipe]-p_old[1])-f_D*dt/(2*D)*abs(v_old[1])*v_old[1] \
        +dt*g*np.sin(alpha)


    pipe.set_boundary_conditions_next_timestep(v_boundary_res[it_pipe],p_boundary_res[it_pipe],v_boundary_tur[it_pipe])
    p_boundary_tur[it_pipe] = pipe.p_boundary_tur

    pipe.timestep_characteristic_method()


    # lo_00.remove()
    # lo_01.remove()
    # lo_02.remove()
    # lo_00, = axs2[0].plot(pl_vec,pressure_conversion(pipe.p_old,'Pa','mWS')[0],marker='.',c='blue')
    # lo_01, = axs2[1].plot(pl_vec,pipe.v_old,marker='.',c='blue')
    # lo_02, = axs2[2].plot(level_vec_2,c='blue')
    # fig2.suptitle(str(it_pipe))
    # fig2.canvas.draw()
    # fig2.canvas.flush_events()
    # fig2.tight_layout()
    # plt.pause(0.1)    

    p_old = pipe.p_old
    v_old = pipe.v_old  

        
        

In [11]:
%matplotlib qt5
fig1,axs1 = plt.subplots(3,2)
axs1[0,0].plot(t_vec,pressure_conversion(p_boundary_res,'Pa','mWS')[0])
axs1[0,1].plot(t_vec,v_boundary_res)
axs1[1,0].plot(t_vec,pressure_conversion(p_boundary_tur,'Pa','mWS')[0])
axs1[1,1].plot(t_vec,v_boundary_tur)
axs1[2,0].plot(t_vec,level_vec)
axs1[0,0].set_title('Pressure Reservoir')
axs1[0,1].set_title('Velocity Reservoir')
axs1[1,0].set_title('Pressure Turbine')
axs1[1,1].set_title('Velocity Turbine')
axs1[0,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[0,0].set_ylabel(r'$p$ [mWS]')
axs1[0,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[0,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs1[1,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[1,0].set_ylabel(r'$p$ [mWS]')
axs1[1,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[1,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
fig1.tight_layout()
plt.show()